## 1. Setup

### 1.1 Installing dependencies

In [1]:
## Installing dependencies 
# %pip install tensorflow
# %pip install tensorflow-gpu
# %pip install opencv-python
# %pip install matplotlib

### 1.2 Importing libraries 

In [2]:
from tensorflow.keras.layers import Conv2D,  MaxPooling2D, MaxPool2D,Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from matplotlib import pyplot as plt
from tensorflow import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import imghdr
import cv2
import os

2023-10-14 12:10:59.669095: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-14 12:10:59.669122: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-14 12:10:59.669975: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-14 12:10:59.757616: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 1.3 Defining parameters

In [ ]:
imgHieght = 256
imgWidth = 256
optimizer = keras.optimizers.legacy.Adam(learning_rate=0.01)
lossMethod = keras.losses.categorical_crossentropy
labelMode = 'categorical'
size = 0.05

## 2. Load Data

In [ ]:
## define the path of repective datasets

trainDataPath   = './dataset/train/'
testDataPath    = './dataset/test/'
validationPath  = './dataset/validation/'

### 2.1 Train Data

In [ ]:
## loading complete train data
CompleteTrainDataSet = tf.keras.utils.image_dataset_from_directory(trainDataPath,label_mode=labelMode,image_size=(imgHieght,imgWidth))

## scaling pixel in the range [0,1]
scaledCompleteTrainDataSet = CompleteTrainDataSet.map(lambda x,y:(x/255,y))

## ClassNames
classNames = CompleteTrainDataSet.class_names

## iterator to move from one batch to another 
dataIterator = scaledCompleteTrainDataSet.as_numpy_iterator()


### 2.2 Validation Data

In [ ]:
validationData = tf.keras.utils.image_dataset_from_directory(validationPath,label_mode=labelMode,image_size=(imgHieght,imgWidth))

scaledValidationData = validationData.map(lambda x,y:(x/255,y))

validationData = scaledValidationData

## 3. Data Visulization

In [ ]:
## creating one batch
batch = dataIterator.next()
print("size of one batch is ",batch[0].shape[0])
print("size of image is : ",batch[0].shape[1:])

In [ ]:
## min / max value of pixel
print("Scaling result")
print("max ==>",batch[0].max())
print("min ==>",batch[0].min())

In [ ]:
## Printing Indexes and repective class Names for batch
for i in batch[1][:20]:
    print(i)

In [ ]:
## Data Visualization

plt.figure(figsize=(10, 10))
for images, labels in CompleteTrainDataSet.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    if labelMode == 'int':
      plt.title(CompleteTrainDataSet.class_names[labels[i]])
    else:
      plt.title(CompleteTrainDataSet.class_names[np.argmax(labels[i])])
    plt.axis("off")

## 4. Splitting Data

- **Splitting train data into small part**

    - Splitting Big training data into 5 parts 
    - Total 625 batches [20000 images] are present
    - Making 5 parts of 125 batch each
    - Each batch contains 32 images group
    - Each group contains 4000 images



In [ ]:
groupSize = int(len(CompleteTrainDataSet)*size)

trainData_0 = scaledCompleteTrainDataSet.take(groupSize)
trainData_1 = scaledCompleteTrainDataSet.skip(groupSize).take(groupSize)
trainData_2 = scaledCompleteTrainDataSet.skip(groupSize*2).take(groupSize)
trainData_3 = scaledCompleteTrainDataSet.skip(groupSize*3).take(groupSize)
trainData_4 = scaledCompleteTrainDataSet.skip(groupSize*4).take(groupSize)

In [ ]:
batch_0 = trainData_0.as_numpy_iterator().next()
batch_1 = trainData_1.as_numpy_iterator().next()
batch_2 = trainData_2.as_numpy_iterator().next()
batch_3 = trainData_3.as_numpy_iterator().next()
batch_4 = trainData_4.as_numpy_iterator().next()
print(len(trainData_0))

In [ ]:
batches = [batch_0,batch_1,batch_2,batch_3,batch_4]
for batch in batches:
    print(batch[0].shape)
    print(batch[1].shape)
batch_0[0].max()

## 5. Defining Models

In [ ]:
## Function to evaluate the performance of model

def plotLoss(history):
    fig = plt.figure()
    plt.plot(history.history['loss'],color='teal',label='loss')
    plt.plot(history.history['val_loss'],color='orange',label='val_loss')
    fig.suptitle('loss',fontsize=20)
    plt.legend(loc='upper left')
    plt.show()

def plotAccuracy(history):
    fig = plt.figure()
    plt.plot(history.history['accuracy'],color='teal',label='Accuracy')
    plt.plot(history.history['val_accuracy'],color='orange',label='val_accuracy')
    fig.suptitle('Accuracy',fontsize=20)
    plt.legend(loc='upper left')
    plt.show()

### 5.1 First Model

In [ ]:
## Model 0 

model_0 = Sequential() 

model_0.add(Conv2D(16,(3,3),1,activation='relu',input_shape=(imgHieght,imgWidth,3)))
model_0.add(MaxPool2D())

model_0.add(Conv2D(32,(3,3),1,activation='relu'))
model_0.add(MaxPool2D())

model_0.add(Conv2D(64,(3,3),1,activation='relu'))
model_0.add(MaxPool2D())

model_0.add(Flatten())

model_0.add(Dense(256,activation='relu'))
model_0.add(Dense(4,activation='sigmoid'))

In [ ]:
model_0.compile(optimizer,loss=lossMethod,metrics=['accuracy'])
model_0.summary()

In [ ]:
hist_0 = model_0.fit(trainData_0,epochs=5,validation_data=validationData)

In [ ]:
plotLoss(hist_0)

In [ ]:
plotAccuracy(hist_0)

### 5.2 Second Model

In [ ]:
model_1 = Sequential()

model_1.add(Conv2D(16,(3,3),1,activation='relu',input_shape=(imgHieght,imgWidth,3)))
model_1.add(MaxPool2D())

model_1.add(Conv2D(32,(3,3),1,activation='relu'))
model_1.add(MaxPool2D())

model_1.add(Conv2D(64,(3,3),1,activation='relu'))
model_1.add(MaxPool2D())

model_1.add(Flatten())

model_1.add(Dense(256,activation='relu'))
model_1.add(Dense(128,activation='relu'))
model_1.add(Dense(4,activation='softmax'))

model_1.compile(optimizer=optimizer,loss=lossMethod,metrics=['accuracy'])
model_1.summary()

In [ ]:
hist_1 = model_1.fit(trainData_1,epochs=5,validation_data=validationData)

In [ ]:
plotLoss(hist_1)

In [ ]:
plotAccuracy(hist_1)

### 5.3 Third Model

In [ ]:
model_2 = Sequential()

model_2.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(imgHieght,imgWidth, 3)))
model_2.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model_2.add(MaxPooling2D(pool_size=(2, 2), name='MaxPool1'))
model_2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model_2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model_2.add(MaxPooling2D(pool_size=(2, 2), name='MaxPool2'))
model_2.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model_2.add(MaxPooling2D(pool_size=(2, 2), name='MaxPool3'))

# Flatten the output from Conv5-3 to generate a feature vector
model_2.add(Flatten())
model_2.add(Dense(256, activation='relu'))
model_2.add(Dense(4, activation='relu'))

model_2.compile(optimizer=optimizer, loss=lossMethod, metrics=['accuracy'])
model_2.summary()

hist_2 = model_2.fit(trainData_2,epochs=10,validation_data=validationData)
plotLoss(hist_2)
plotAccuracy(hist_2)


## 6. Pretrained Model 

In [ ]:
def trainModel(trainedModel,train_data):

    for layers in trainedModel.layers:
        layers.trainable = False

    # trainedModel.summary()

    model = Sequential()

    model.add(trainedModel)

    model.add(Flatten())
    model.add(Dense(4,activation='softmax'))

    # print(model.summary())
    model.compile(optimizer=optimizer,loss=lossMethod,metrics=['accuracy'])

    history = model.fit(train_data,epochs=4,validation_data=validationData)

    plotLoss(history)
    plotAccuracy(history)

### 6.1 VGG16

In [ ]:
VGG16_Model = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(imgHieght,imgWidth,3),
    classes=4
)

In [ ]:
# trainModel(VGG16_Model, trainData_3)
for layers in VGG16_Model.layers:
    layers.trainable = False

    # trainedModel.summary()

    model = Sequential()

    model.add(VGG16_Model)

    model.add(Flatten())
    model.add(Dense(4096,activation='softmax'))
    model.add(Dense(4,activation='softmax'))

    # print(model.summary())
    model.compile(optimizer=optimizer,loss=lossMethod,metrics=['accuracy'])

    history = model.fit(trainData_4,epochs=4,validation_data=validationData)

    plotLoss(history)
    plotAccuracy(history)

### 6.2 ResNet50

In [ ]:
ResNet50_Model= tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(imgHieght,imgWidth,3),
    classes=4
)

In [ ]:
trainModel(ResNet50_Model,trainData_2)

## 7. Prediction

In [ ]:
predictionsMap = {}
for image in os.listdir(testDataPath):

    imagePath = os.path.join(testDataPath,image)

    img = tf.keras.utils.load_img(
        imagePath, target_size=(imgHieght,imgWidth)
    )

    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = model_0.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    predictionsMap[imagePath] = classNames[np.argmax(score)]  

    # print(
    #     "This image most likely belongs to {} with a {:.2f} percent confidence."
    #     .format(classNames[np.argmax(score)], 100 * np.max(score))
    # )
print(predictionsMap)